<a href="https://colab.research.google.com/github/krishnannarayanaswamy/GenAI-Relevance-LLM-demo/blob/main/Term_Based_Search_%26_Vector_Search_with_DataStax_Astra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install cassandra-driver openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.3 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY=
ASTRA_DB_SECURE_BUNDLE_PATH=
ASTRA_DB_CLIENT_ID=
ASTRA_DB_CLIENT_SECRET=
ASTRA_DB_APPLICATION_TOKEN=
ASTRA_DB_KEYSPACE=
ASTRA_DB_ID=
ASTRA_DB_REGION=

In [ ]:
# Astra Connection (Based on cqlsession, from cassio library)
import os
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

def getCQLSession(mode='astra_db'):
    print('Initializing CQL Session')
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_CLIENT_SECRET,
            ),
        )
        astraSession = cluster.connect()
        astraSession.row_factory = dict_factory
        print('Connected')
        return astraSession
    elif mode == 'local':
        cluster = Cluster()
        localSession = cluster.connect()
        return localSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    return ASTRA_DB_KEYSPACE


In [ ]:
# Open the cennection with Astra
session = getCQLSession()

Initializing CQL Session


ERROR:cassandra.connection:Closing connection <AsyncoreConnection(138531249566384) 168ca433-d1dd-4e62-a907-f68b79b2c042-ap-south-1.db.astra.datastax.com:29042:914a1012-1ca7-469b-bb97-30bfcd46efad> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Connected


In [ ]:
keyspace = ASTRA_DB_KEYSPACE
table_user = "tbl_users"
table_post = "tbl_post"

In [ ]:
# More functions used on the code
import openai
openai.api_key = OPENAI_API_KEY
def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = openai.Embedding.create(model=model, input=text)
    return response.data[0]['embedding']

In [ ]:
    #query english embeddings with english prompt
from cassandra.query import SimpleStatement
import pandas as pd
import numpy as np

query = SimpleStatement(
    f"""
    SELECT user_id, username, fullname
    FROM {keyspace}.{table_user}
    where fullname : 'Sangita' or username : 'Sangita'
    """
    )

results = session.execute(query)

top_products = results._current_rows
for row in top_products:
  print(f"""{row["user_id"]}, {row["fullname"]}, {row["username"]}\n""")

e2a6efaf-55f5-419a-91a2-82bdf7f20564, Sangita Devi, sd920671

dca92e26-c593-4417-916b-a6405d4b296d, Sangita Panwar, sangitapanwar150

54f5bfa8-1c9e-42c2-8d4f-59173f4ed128, Sangita Modak, sangitamodak2002

565e41e9-23ad-4107-9056-96903820145a, Sangita Dutta, sangitadutta98320

8c19ec2c-a4bb-4159-a65b-19ca77173469, Sangita lokhande, lokhandesangita123

d1147c7b-1053-4408-a2b5-eb49d41aa848, Sangita Chopra, sangitanoni



In [ ]:
#query english embeddings with english prompt
from cassandra.query import SimpleStatement
import pandas as pd
import numpy as np

keyword = "reelstar"
embedding = generate_embedding(keyword)

query = SimpleStatement(
    f"""
    SELECT post_id, description
    FROM {keyspace}.{table_post}
    ORDER BY openai_embedding ANN OF {embedding}
    LIMIT 10"""
    )

results = session.execute(query)

top_products = results._current_rows
for row in top_products:
  print(f"""{row["post_id"]}, {row["description"]}\n""")

181afa04-2f80-4f9f-b1f2-02f98c565964, Reelstar

9596a3cb-5cb7-4d39-8ef5-49950b942454, Reelstar

462dc6e5-cc90-40b8-8011-c654d5dd4e9c, ReelStar

675d36e8-c353-4a85-b934-f59791214a2e, ReelStar

36d51a7d-8f17-48ec-ae41-2f91b0c4b1d1, reel star

3867bf34-8adc-4c5d-a6c8-5e1d8e8ba9eb, reel star

96f1ebdf-2e83-468c-9bf7-99d766cd4cd9, reel star

f34b7f62-3564-4430-9603-765b9ef4186d, reel star

60e0a2a7-0ff4-462f-908b-a5e5b0c24671, @reelstar

26395517-1a1e-4c29-a3b2-c58b7588ccde, Reelstar#reelstar

